In [12]:
from pprint import pprint
import os
import sys

your_cluster_name = "mustafa7"

## The private key pair for accessing cluster.
private_key = "/home/mustafa/interns_oregon_key.pem"

## If delete cfncluster after job is done./
delete_cfncluster= False

print("variables set")

variables set


In [13]:
sys.path.append("../../src/cirrus_ngs")
from cfnCluster import CFNClusterManager, ConnectionManager
from util import AddonsManager

## Create a new cluster
master_ip_address = CFNClusterManager.create_cfn_cluster(cluster_name=your_cluster_name)
ssh_client = ConnectionManager.connect_master(hostname=master_ip_address,
               username="ec2-user",
               private_key_file=private_key)

cluster mustafa7 does exist.

Status: CREATE_COMPLETE
Status: CREATE_COMPLETE
MasterServer: RUNNING
MasterServer: RUNNING
Output:"MasterPublicIP"="34.214.180.149"
Output:"MasterPrivateIP"="172.31.33.31"
Output:"GangliaPublicURL"="http://34.214.180.149/ganglia/"
Output:"GangliaPrivateURL"="http://172.31.33.31/ganglia/"

connecting
connected


### Check Scripts on Cluster

This section containss cells that can be used to check which scripts are currently on the cluster being used. 

#### Get all supported pipelines

In [14]:
#This cell must be run before others in this section

scripts = AddonsManager.get_scripts_dict(ssh_client)
print()
print("Supported Pipelines:", AddonsManager.get_all_pipeline_names(scripts))

Executing python /shared/workspace/Pipelines/util/GetScripts.py

Supported Pipelines: ['RNASeq', 'ChiPSeq', 'DNASeq', 'SmallRNASeq']


#### Check which workflows are in a given pipeline

In [16]:
#can be set to a supported pipeline name or "all"
target_pipeline = "DNASeq"

print("Supported Workflows in {} Pipeline(s): ".format(target_pipeline), end="")
pprint(AddonsManager.get_workflows_in_pipeline(scripts, target_pipeline), indent=2)

Supported Workflows in DNASeq Pipeline(s): ['bwa_gatk', 'bwa_mutect']


#### Check which shell scripts are in a given pipeline or workflow

In [19]:
#can be set to a supported pipeline or "all"
target_pipeline = "DNASeq"

#can be set to a support workflow or "all"
#if target_pipeline == "all" then this variable is ignored
target_workflow = "bwa_gatk"

pprint(AddonsManager.get_scripts(scripts, target_pipeline, target_workflow))

{'All DNASeq Workflows': ['merge.sh',
                          'split.sh',
                          'sort.sh',
                          'bwa.sh',
                          'post.sh',
                          'dedup.sh'],
 'All Pipelines': ['fastqc.sh',
                   'multiqc.sh',
                   'trim.sh',
                   'run.sh',
                   'merge_counts.sh'],
 'bwa_gatk': ['haplo.sh', 'group_vcf.sh', 'filter.sh']}


#### Check which step calls a script

In [21]:
#should be the name of a shell script on the cluster (include sh extension)
target_script = "fastqc.sh"

print(AddonsManager.get_steps_calling_script(ssh_client, scripts, target_script))

Executing cat /shared/workspace/Pipelines/config/tools.yaml
fastqc.sh called from:
fastqc in all Pipelines
kallisto_fastqc in all Pipelines
star_gatk_fastqc in all Pipelines
star_htseq_fastqc in all Pipelines
star_rsem_fastqc in all Pipelines



#### Print out a script

In [31]:
#all targets must be set to a valid pipeline/workflow/script
#use cells above to check valid options
target_pipeline = "ChiPSeq"
target_workflow = "homer"
target_script = "bowtie.sh"


loc, file_cat = AddonsManager.cat_script(ssh_client, scripts, target_pipeline, target_workflow, target_script)
print("".join(["#"]*len(loc)) + "\n{}\n".format(loc) + "".join(["#"]*len(loc)))
print()
print(file_cat)

Executing cat /shared/workspace/Pipelines/scripts/ChiPSeq/homer/bowtie.sh
#################
Workflow Specific
#################

#!/bin/bash

project_name=$1
workflow=$2
file_suffix=$3  #extension of input file, does not include .gz if present in input
root_dir=$4
fastq_end1=$5
fastq_end2=$6
input_address=$7    #this is an s3 address e.g. s3://path/to/input/directory
output_address=$8   #this is an s3 address e.g. s3://path/to/output/directory
log_dir=$9
is_zipped=${10}    #either "True" or "False", indicates whether input is gzipped
num_threads=${11}

#logging
log_dir=$log_dir/$fastq_end1
mkdir -p $log_dir
log_file=$log_dir/'bowtie.log'
exec 1>>$log_file
exec 2>>$log_file

status_file=$log_dir/'status.log'
touch $status_file

#prepare output directories
workspace=$root_dir/$project_name/$workflow/$fastq_end1
mkdir -p $workspace

echo "%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%"
date
echo "%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%"

check_step_already_done $JOB_NAME $status_file


##DOWNLOAD##
if

### Check Software on Cluster

This section is used to check what software is installed on the cluster.

#### Get all software installed on cluster

In [25]:
software = AddonsManager.get_software_dict(ssh_client)
print("All Installed Software: ")
pprint(software)

Executing python /shared/workspace/Pipelines/util/GetSoftware.py
All Installed Software: 
{'FastQC': ['0.11.3'],
 'HTSeq': ['0.9.1 (installed with pip)'],
 'MultiQC': ['1.3 (installed with conda)'],
 'RSEM': ['1.3.0'],
 'STAR': ['2.3.0e', '2.5.1a', '2.5.3a'],
 'Trimmomatic': ['0.36'],
 'bedtools2': ['2.19.1'],
 'blat': ['36x1'],
 'bowtie': ['1.0.1'],
 'bowtie2': ['2.3.2-legacy'],
 'bwa': ['0.7.12-r1039'],
 'cutadapt': ['1.14 (installed with conda)'],
 'gatk': ['3.8-0'],
 'ghostscript': ['9.19'],
 'homer': ['4.8.3'],
 'java': ['jre1.8.0_144'],
 'kallisto': ['0.43.1'],
 'picard': ['1.96'],
 'sambamba': ['0.4.7'],
 'samblaster': ['0.1.21'],
 'samtools': ['1.1'],
 'tabix': ['0.2.6'],
 'vcftools': ['0.1.12b'],
 'weblogo': ['2.8']}


#### Check the version of a specific tool

In [28]:
#ignores case
target_tool = "htseq"

print(AddonsManager.check_tool_is_installed(software, target_tool))

HTSeq installed with version(s):
0.9.1 (installed with pip)
